In [1]:
# ============================================
# 1. Importing Modules
# ============================================
import os
import string
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from datetime import datetime

# reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# make results dir
os.makedirs("results", exist_ok=True)


Using device: cuda


In [2]:
# ============================================
# 2. Dataset Loading (fixed char maps + robust label parsing)
# ============================================
class OCRDataset(Dataset):
    def __init__(self, labels_file, images_dir, transform=None, characters=None):
        """
        labels_file: each line -> "filename label" (label may contain spaces)
        images_dir: directory containing images referenced in labels_file
        transform: torchvision transforms to apply
        characters: optional string of valid characters (keeps mapping stable)
        """
        self.samples = []
        self.transform = transform

        if characters is None:
            # you can customize which characters you want to support
            characters = string.ascii_letters + string.digits + string.punctuation + " "
        self.characters = characters

        # Mapping: reserve 0 for CTC blank / padding
        self.char_to_num = {c: i + 1 for i, c in enumerate(self.characters)}
        self.num_to_char = {i + 1: c for i, c in enumerate(self.characters)}
        self.num_to_char[0] = ""  # blank / padding

        if not os.path.exists(labels_file):
            raise FileNotFoundError(f"Labels file not found: {labels_file}")

        with open(labels_file, "r", encoding="utf-8") as f:
            for line in f:
                line = line.rstrip("\n")
                if line == "":
                    continue
                parts = line.strip().split(maxsplit=1)
                if len(parts) == 0:
                    continue
                if len(parts) == 1:
                    filename = parts[0]
                    label = ""
                else:
                    filename, label = parts
                img_path = os.path.join(images_dir, filename)
                if os.path.exists(img_path) and label.strip() != "":
                    self.samples.append((img_path, label))
                # else: skip missing files or empty labels

    def encode_text(self, text):
        # unknown characters map to 0 (blank/padding) — but better to remove unknown chars
        return torch.tensor([self.char_to_num.get(c, 0) for c in text], dtype=torch.long)

    def decode_indices(self, indices):
        # indices: sequence of ints
        return "".join([self.num_to_char.get(int(i), "") for i in indices if i != 0])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert("L")  # grayscale
        if self.transform:
            img = self.transform(img)
        label_encoded = self.encode_text(label)
        return img, label_encoded, label  # also return original label string for convenience


In [3]:
# ============================================
# 3. Preprocessing of Dataset + collate_fn (CTC target handling fixed)
# ============================================
IMG_HEIGHT = 32
IMG_WIDTH = 128
BATCH_SIZE = 8

# Data augmentation and transforms (stronger)
train_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomAffine(degrees=3, translate=(0.02,0.02), shear=2, scale=(0.95,1.05), fill=255),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.5),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# character set — keep it stable and reasonably small if you have limited labels
characters = string.ascii_letters + string.digits + string.punctuation + " "

train_dataset = OCRDataset(
    labels_file=r"C:\Users\acer\OneDrive\Desktop\Handwriting_recognition\data_split_reduced\train\labels.txt",
    images_dir=r"C:\Users\acer\OneDrive\Desktop\Handwriting_recognition\data_split_reduced\train\images",
    transform=train_transform,
    characters=characters
)

val_dataset = OCRDataset(
    labels_file=r"C:\Users\acer\OneDrive\Desktop\Handwriting_recognition\data_split_reduced\val\labels.txt",
    images_dir=r"C:\Users\acer\OneDrive\Desktop\Handwriting_recognition\data_split_reduced\val\images",
    transform=val_transform,
    characters=characters
)

def collate_fn(batch):
    """
    Returns:
      imgs: tensor (B, 1, H, W)
      targets: 1D tensor concatenating all label indices (for CTC)
      target_lengths: tensor (B,) lengths of each label
      labels_padded: padded labels (B, max_len) -- useful for reconstructing true_texts
      raw_texts: list of original label strings
    """
    imgs = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    raw_texts = [item[2] for item in batch]

    imgs = torch.stack(imgs, 0)
    label_lengths = torch.tensor([len(l) for l in labels], dtype=torch.long)

    # create padded labels for reconstruction/printing
    if len(labels) == 0:
        return imgs, torch.tensor([], dtype=torch.long), label_lengths, torch.tensor([], dtype=torch.long), raw_texts

    labels_padded = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)  # (B, Lmax)

    # flatten targets for CTCLoss: 1D concatenation of all labels (no padding)
    targets = torch.cat([l for l in labels]).to(torch.long)  # 1D

    return imgs, targets, label_lengths, labels_padded, raw_texts

# ============================================
# 3b. DataLoader creation (DEBUG-FRIENDLY / Windows-safe)
# Replace your previous train_loader / val_loader definitions with these.
# ============================================
# Note: use num_workers=0 on Windows or with OneDrive paths to avoid hangs.
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0,        # <-- IMPORTANT: 0 avoids worker deadlocks on Windows / cloud drives
    pin_memory=False,     # safer when num_workers=0 and avoids some platform issues
    persistent_workers=False,
    timeout=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,        # <-- keep 0 for safety
    pin_memory=False,
    persistent_workers=False,
    timeout=0
)

print("Train loader and Val loader created. num_workers=0, pin_memory=False")
print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")


Train loader and Val loader created. num_workers=0, pin_memory=False
Train batches: 1250, Val batches: 250


In [4]:
# ============================================
# 4. Define Model (Improved CRNN + CTC)
# ============================================
class CRNN(nn.Module):
    def __init__(self, img_h, num_classes, dropout=0.2):
        super(CRNN, self).__init__()
        # conv blocks
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),  # H/2

            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.MaxPool2d(2,2),  # H/4

            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            # keep width resolution; no further pooling in height to preserve sequence length
        )
        self.img_h = img_h
        # RNN: input size = (channels_after_cnn * (H//4))
        rnn_input_size = (img_h // 4) * 128
        self.rnn = nn.LSTM(input_size=rnn_input_size, hidden_size=256, num_layers=2, bidirectional=True, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(256*2, num_classes + 1)  # +1 for CTC blank
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (B, 1, H, W)
        x = self.cnn(x)  # (B, C, H', W')
        b, c, h, w = x.size()
        # collapse height and channels into features per time-step (width dimension is time)
        x = x.permute(0, 3, 1, 2)  # (B, W', C, H')
        x = x.contiguous().view(b, w, c * h)  # (B, W', C*H')
        x, _ = self.rnn(x)  # (B, W', 2*hidden)
        x = self.dropout(x)
        x = self.fc(x)  # (B, W', num_classes+1)
        x = x.log_softmax(2)  # for CTC Loss
        return x

num_classes = len(characters)
model = CRNN(IMG_HEIGHT, num_classes).to(device)
print(model)


CRNN(
  (cnn): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
  )
  (rnn): LSTM(1024, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=512, out_features=96, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [5]:
# ============================================
# 5 (debugged) Train Model — safe & verbose (copy to replace your Block 5)
# ============================================
optimizer = optim.Adam(model.parameters(), lr=1e-3)
ctc_loss_fn = nn.CTCLoss(blank=0, zero_infinity=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

EPOCHS = 30
train_losses, val_losses = [], []
best_val_loss = float("inf")
best_model_path = "results/best_ocr_model.pth"

for epoch in range(EPOCHS):
    print(f"\n===== Epoch {epoch+1}/{EPOCHS} =====")
    model.train()
    running_train_loss = 0.0
    batch_count = 0

    # ---------- TRAINING LOOP ----------
    try:
        # wrap iteration in try so we catch loader errors immediately
        for batch_idx, batch in enumerate(train_loader):
            batch_count += 1

            # Debug: show that we successfully retrieved a batch
            if batch_idx == 0:
                print("✅ Successfully fetched first training batch from DataLoader.")

            # Unpack depending on collate format (keeps compatibility)
            # Our collate_fn returns: imgs, targets, target_lengths, labels_padded, raw_texts
            if len(batch) == 5:
                imgs, targets, target_lengths, labels_padded, raw_texts = batch
            elif len(batch) == 4:
                imgs, targets, target_lengths, labels_padded = batch
                raw_texts = [""] * imgs.size(0)
            elif len(batch) == 3:
                imgs, targets, target_lengths = batch
                labels_padded = None
                raw_texts = [""] * imgs.size(0)
            else:
                raise ValueError(f"Unexpected batch tuple length: {len(batch)}")

            imgs = imgs.to(device)
            targets = targets.to(device)
            target_lengths = target_lengths.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)              # (B, W, C)
            outputs = outputs.permute(1, 0, 2) # (W, B, C) for CTCLoss

            input_lengths = torch.full(
                (outputs.size(1),),
                outputs.size(0),
                dtype=torch.long
            ).to(device)

            loss = ctc_loss_fn(outputs, targets, input_lengths, target_lengths)

            if torch.isnan(loss) or torch.isinf(loss):
                print(f"⚠️ Skipping batch {batch_idx} due to NaN/Inf loss.")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()

            running_train_loss += loss.item()

            # periodic print
            if (batch_idx + 1) % 50 == 0:
                avg_loss = running_train_loss / (batch_idx + 1)
                print(f"Batch {batch_idx+1}/{len(train_loader)} - Avg Train Loss: {avg_loss:.4f}")

    except Exception as e:
        # If DataLoader worker crashed or another error happened, print full stack and re-raise
        import traceback
        print("❌ Exception while iterating train_loader:")
        traceback.print_exc()
        raise e

    avg_train_loss = running_train_loss / max(1, batch_count)
    train_losses.append(avg_train_loss)

    # ---------- VALIDATION LOOP ----------
    model.eval()
    running_val_loss = 0.0
    val_batch_count = 0
    try:
        with torch.no_grad():
            for batch_idx, batch in enumerate(val_loader):
                val_batch_count += 1
                if len(batch) == 5:
                    imgs, targets, target_lengths, labels_padded, raw_texts = batch
                elif len(batch) == 4:
                    imgs, targets, target_lengths, labels_padded = batch
                    raw_texts = [""] * imgs.size(0)
                elif len(batch) == 3:
                    imgs, targets, target_lengths = batch
                    labels_padded = None
                    raw_texts = [""] * imgs.size(0)
                else:
                    continue

                imgs = imgs.to(device)
                targets = targets.to(device)
                target_lengths = target_lengths.to(device)

                outputs = model(imgs)
                outputs = outputs.permute(1, 0, 2)
                input_lengths = torch.full(
                    (outputs.size(1),),
                    outputs.size(0),
                    dtype=torch.long
                ).to(device)
                loss = ctc_loss_fn(outputs, targets, input_lengths, target_lengths)
                running_val_loss += loss.item()
    except Exception as e:
        import traceback
        print("❌ Exception while iterating val_loader:")
        traceback.print_exc()
        raise e

    avg_val_loss = running_val_loss / max(1, val_batch_count)
    val_losses.append(avg_val_loss)

    # scheduler & lr print
    prev_lr = optimizer.param_groups[0]['lr']
    scheduler.step(avg_val_loss)
    new_lr = optimizer.param_groups[0]['lr']
    if new_lr != prev_lr:
        print(f"🔻 LR reduced from {prev_lr:.6f} to {new_lr:.6f}")

    print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f} | LR: {new_lr:.6f}")

    # ---------- SAVE BEST ----------
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "val_loss": best_val_loss
        }, best_model_path)
        print(f"💾 Saved best model to {best_model_path}")

# final save
torch.save(model.state_dict(), "results/ocr_model_final.pth")
print("✅ Training complete. Final model saved to 'results/ocr_model_final.pth'")



===== Epoch 1/30 =====
✅ Successfully fetched first training batch from DataLoader.
Batch 50/1250 - Avg Train Loss: 5.6343
Batch 100/1250 - Avg Train Loss: 4.7587
Batch 150/1250 - Avg Train Loss: 4.3796
Batch 200/1250 - Avg Train Loss: 4.1355
Batch 250/1250 - Avg Train Loss: 3.9871
Batch 300/1250 - Avg Train Loss: 3.8637
Batch 350/1250 - Avg Train Loss: 3.7585
Batch 400/1250 - Avg Train Loss: 3.6963
Batch 450/1250 - Avg Train Loss: 3.6282
Batch 500/1250 - Avg Train Loss: 3.5785
Batch 550/1250 - Avg Train Loss: 3.5245
Batch 600/1250 - Avg Train Loss: 3.4788
Batch 650/1250 - Avg Train Loss: 3.4338
Batch 700/1250 - Avg Train Loss: 3.4018
Batch 750/1250 - Avg Train Loss: 3.3740
Batch 800/1250 - Avg Train Loss: 3.3439
Batch 850/1250 - Avg Train Loss: 3.3168
Batch 900/1250 - Avg Train Loss: 3.2968
Batch 950/1250 - Avg Train Loss: 3.2728
Batch 1000/1250 - Avg Train Loss: 3.2502
Batch 1050/1250 - Avg Train Loss: 3.2240
Batch 1100/1250 - Avg Train Loss: 3.2021
Batch 1150/1250 - Avg Train Loss:

In [6]:
# ============================================
# 6. Greedy decoder and sample predictions (robust)
# ============================================
def greedy_decoder(output, num_to_char):
    """
    Greedy CTC decoder: input output is (B, W, C) with log_softmax applied.
    returns list of decoded strings (one per batch item)
    """
    # output: (B, W, C)
    preds = torch.argmax(output, 2).cpu().numpy()  # (B, W)
    texts = []
    for o in preds:
        text = ""
        prev = 0
        for idx in o:
            if idx != prev and idx != 0:
                text += num_to_char.get(int(idx), "")
            prev = idx
        texts.append(text)
    return texts

# mapping consistent with datasets
num_to_char = val_dataset.num_to_char
char_to_num = val_dataset.char_to_num

# run on one batch and print/save few samples
model.eval()
imgs, targets, target_lengths, labels_padded, raw_texts = next(iter(val_loader))
imgs_cuda = imgs.to(device)
with torch.no_grad():
    outputs = model(imgs_cuda)  # (B, W, C)
    decoded_texts = greedy_decoder(outputs, num_to_char)

# Reconstruct true_texts from labels_padded and target_lengths (padded)
true_texts = []
for row_idx in range(labels_padded.size(0)):
    length = int(target_lengths[row_idx])
    row = labels_padded[row_idx][:length].cpu().numpy()
    true_texts.append("".join([num_to_char.get(int(x), "") for x in row]))

# Save sample images with GT and Pred text
for i in range(min(8, imgs.size(0))):
    arr = imgs[i].cpu().squeeze().numpy()
    plt.figure(figsize=(4,1.5))
    plt.imshow(arr, cmap="gray")
    plt.axis("off")
    plt.title(f"GT: {true_texts[i]}\nPred: {decoded_texts[i]}")
    fname = f"results/sample_pred_{i}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    plt.savefig(fname, bbox_inches='tight', dpi=150)
    plt.close()
    print(f"Saved sample {i} -> {fname}")


Saved sample 0 -> results/sample_pred_0_20251015_131450.png
Saved sample 1 -> results/sample_pred_1_20251015_131450.png
Saved sample 2 -> results/sample_pred_2_20251015_131451.png
Saved sample 3 -> results/sample_pred_3_20251015_131451.png
Saved sample 4 -> results/sample_pred_4_20251015_131451.png
Saved sample 5 -> results/sample_pred_5_20251015_131451.png
Saved sample 6 -> results/sample_pred_6_20251015_131451.png
Saved sample 7 -> results/sample_pred_7_20251015_131451.png


In [7]:
# ============================================
# 7. Compute character-level Precision / Recall / F1 and exact-match accuracy (and save bar graph)
# ============================================
def char_level_prf(preds, trues):
    """
    Simple character-level TP/FP/FN counting using position alignment:
      - For positions where both exist: if equal -> TP; else -> FP and FN (substitution)
      - If predicted only -> FP
      - If true only -> FN
    returns precision, recall, f1
    """
    TP = 0
    FP = 0
    FN = 0
    for p, t in zip(preds, trues):
        lp = len(p)
        lt = len(t)
        L = max(lp, lt)
        for i in range(L):
            has_p = i < lp
            has_t = i < lt
            if has_p and has_t:
                if p[i] == t[i]:
                    TP += 1
                else:
                    FP += 1
                    FN += 1
            elif has_p and (not has_t):
                FP += 1
            elif has_t and (not has_p):
                FN += 1
    eps = 1e-8
    precision = TP / (TP + FP + eps)
    recall = TP / (TP + FN + eps)
    f1 = 2 * precision * recall / (precision + recall + eps)
    return precision, recall, f1

# Build predictions and ground truths for entire validation set (may be slow if val set large)
all_preds = []
all_trues = []
model.eval()
with torch.no_grad():
    for imgs, targets, target_lengths, labels_padded, raw_texts in val_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)  # (B,W,C)
        batch_preds = greedy_decoder(outputs, num_to_char)
        # reconstruct trues from labels_padded and lengths
        for row_idx in range(labels_padded.size(0)):
            length = int(target_lengths[row_idx])
            row = labels_padded[row_idx][:length].cpu().numpy()
            true_text = "".join([num_to_char.get(int(x), "") for x in row])
            all_trues.append(true_text)
        all_preds.extend(batch_preds)

# compute exact match accuracy
exact_matches = sum([1 for p,t in zip(all_preds, all_trues) if p == t])
accuracy = exact_matches / max(1, len(all_trues))
precision, recall, f1 = char_level_prf(all_preds, all_trues)
print(f"Validation Exact-match Accuracy: {accuracy:.4f}")
print(f"Character-level Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# save bar graph for precision, recall, f1
plt.figure(figsize=(6,4))
plt.bar(["Precision","Recall","F1"], [precision, recall, f1])
plt.ylim(0,1)
plt.title("Character-level Metrics")
plt.ylabel("Score")
plt.savefig("results/precision_recall_f1.png", bbox_inches='tight', dpi=150)
plt.close()
print("Saved results/precision_recall_f1.png")


Validation Exact-match Accuracy: 0.6330
Character-level Precision: 0.7612, Recall: 0.7355, F1: 0.7482
Saved results/precision_recall_f1.png


In [8]:
# ============================================
# 8. Save accuracy and loss graphs (and to disk)
# ============================================
# Loss curves (train & val)
plt.figure(figsize=(10,4))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curves")
plt.legend()
plt.grid(True)
plt.savefig("results/loss_curve.png", bbox_inches='tight', dpi=150)
plt.close()
print("Saved results/loss_curve.png")

# Accuracy (we only computed final val exact-match) -> create a simple line for demonstration
# If you want per-epoch accuracy, you'd compute during training similarly to val loss aggregation per epoch.
plt.figure(figsize=(6,4))
plt.plot([accuracy], marker='o')  # single-point placeholder (replace with per-epoch values if computed)
plt.title("Validation Exact-match Accuracy (final)")
plt.ylim(0,1)
plt.savefig("results/accuracy_point.png", bbox_inches='tight', dpi=150)
plt.close()
print("Saved results/accuracy_point.png")


Saved results/loss_curve.png
Saved results/accuracy_point.png


In [9]:
# ============================================
# 9. Save the final mapping and metadata
# ============================================
import json
meta = {
    "characters": characters,
    "num_classes": num_classes,
    "img_height": IMG_HEIGHT,
    "img_width": IMG_WIDTH,
    "seed": SEED,
    "best_val_loss": float(best_val_loss),
    "saved_at": datetime.now().isoformat()
}
with open("results/metadata.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2)
print("Saved results/metadata.json")


Saved results/metadata.json


In [10]:
# ============================================
# 10. Notes & tips (print for convenience)
# ============================================
print("Done. Files saved into the results/ folder:")
for fname in sorted(os.listdir("results")):
    print("  -", fname)
print("\nTips if performance still low:")
print(" - Increase dataset size or use synthetic augmentation.")
print(" - If sequences are longer than model capacity, increase IMG_WIDTH or reduce pooling.")
print(" - Try a CRNN variant with deeper RNN or transformer-based sequence head if needed.")
print(" - Consider using beam search decoder (with language model) for better final predictions.")


Done. Files saved into the results/ folder:
  - accuracy_point.png
  - best_ocr_model.pth
  - loss_curve.png
  - metadata.json
  - ocr_model_final.pth
  - precision_recall_f1.png
  - sample_pred_0_20251015_131450.png
  - sample_pred_1_20251015_131450.png
  - sample_pred_2_20251015_131451.png
  - sample_pred_3_20251015_131451.png
  - sample_pred_4_20251015_131451.png
  - sample_pred_5_20251015_131451.png
  - sample_pred_6_20251015_131451.png
  - sample_pred_7_20251015_131451.png

Tips if performance still low:
 - Increase dataset size or use synthetic augmentation.
 - If sequences are longer than model capacity, increase IMG_WIDTH or reduce pooling.
 - Try a CRNN variant with deeper RNN or transformer-based sequence head if needed.
 - Consider using beam search decoder (with language model) for better final predictions.
